
# 采用变分算法重构半导体双量子点单-三重态量子门

## 单量子比特情况

## 采用脉冲强度不变，持续时间变的方法。

In [1]:
import numpy as np
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()

def _matrix_0(coeff):
    return expm(-1j*(0*s_z+s_x)*coeff)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(0*s_z+s_x)


def _matrix_1(coeff):
    return expm(-1j*(1*s_z+s_x)*coeff)

def _diff_matrix_1(coeff):
    return -1j*_matrix_1(coeff)@(1*s_z+s_x)


def _matrix_2(coeff):
    return expm(-1j*(2*s_z+s_x)*coeff)

def _diff_matrix_2(coeff):
    return -1j*_matrix_2(coeff)@(2*s_z+s_x)

def _matrix_3(coeff):
    return expm(-1j*(3*s_z+s_x)*coeff)

def _diff_matrix_3(coeff):
    return -1j*_matrix_3(coeff)@(3*s_z+s_x)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_1 = gene_univ_parameterized_gate('gete_1', _matrix_1, _diff_matrix_1)
gate_2 = gene_univ_parameterized_gate('gete_2', _matrix_2, _diff_matrix_2)
gate_3 = gene_univ_parameterized_gate('gete_3', _matrix_3, _diff_matrix_3)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_1('01').on(0)
circ += gate_2('02').on(0)
circ += gate_3('03').on(0)
circ += gate_0('10').on(0)
circ += gate_1('11').on(0)
circ += gate_2('12').on(0)
circ += gate_3('13').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)

grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)

lr = 0.1 
params = 3 * np.random.rand(8)

for i in range(100):
    f, g = grad_ops(params, np.array([[1.],[0.]]), np.array([[0.],[1.]]))
    f, g = grad_ops(params, np.array([[0.],[1.]]), np.array([[1.],[0.]]))
    params = abs(params + lr * np.squeeze(g).real)
    if i % 20 == 0:
        print(f[0,0].real)
        
[theta00, theta01, theta02, theta03, theta10, theta11, theta12, theta13] = params
pr = {'00':theta00, '01':theta01, '02':theta02, '03':theta03, '10':theta10, '11':theta11, '12':theta12, '13':theta13}
state = circ.get_qs(pr=pr)
fid = np.abs(np.vdot(state, [0,1]))

print('\nfid:',fid)
print('\nparams:',params)

0.504964715598844
0.9126772157609289
0.9893397160188581
0.998743377627901
0.9998522498645346

fid: 0.999994275711438

params: [2.19896871 0.08977132 1.10727439 0.63598394 0.6257678  0.58398319
 0.93714334 0.97712595]


只采用 0 和 1 两个分立动作，实现任意单量子比特幺正算符。对多个算符进行验证训练。

In [1]:
import numpy as np
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
import sys
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()

def _matrix_0(coeff):
    return expm(-1j*(0*s_z+s_x)*coeff)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(0*s_z+s_x)


def _matrix_1(coeff):
    return expm(-1j*(1*s_z+s_x)*coeff)

def _diff_matrix_1(coeff):
    return -1j*_matrix_1(coeff)@(1*s_z+s_x)


gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_1 = gene_univ_parameterized_gate('gete_1', _matrix_1, _diff_matrix_1)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_1('01').on(0)
circ += gate_0('10').on(0)
circ += gate_1('11').on(0)
circ += gate_0('20').on(0)
circ += gate_1('21').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)

grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)

lr = 0.05
acc_list = []
for i in range(len(u_mats)):
    params = 2*np.pi* np.random.rand(len(circ.params_name))
    for j in range(len(train_x)):
        f, g = grad_ops(params, train_x[j], train_y[i,j])
        params = abs(params + lr * np.squeeze(g).real)
        
    final_state = []
    for j in range(len(eval_x)):
        sim.reset()
        sim.set_qs(eval_x[j])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
        
        
    acc = np.real(np.mean([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    acc_list.append(acc)
    print("\r", end="")
    print(f"进度: {i+1}/{len(u_mats)} ",  'acc:', acc, end="")
    
print('\n平均保真度为：', np.mean(acc_list))

进度: 100/100  acc: 1.0000000000000004
平均保真度为： 0.9980107389783618


## 采用变强度，不变持续时间的脉冲，实现任意单量子比特门。

In [23]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)

grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)

mean_infid_list = []
max_infid_list = []

lr = 0.05
for i in range(len(u_mats)):
    params = 2*np.pi* np.random.rand(len(circ.params_name))
    for j in range(len(train_x)):
        f, g = grad_ops(params, train_x[j], train_y[i,j])
        params = abs(params + lr * np.squeeze(g).real)
        
    final_state = []
    for j in range(len(eval_x)):
        sim.reset()
        sim.set_qs(eval_x[j])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
        
    mean_infid = 1-np.real(np.mean([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    max_infid = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    mean_infid_list.append(mean_infid)
    max_infid_list.append(max_infid)
#     print("\r", end="")
    print(f"\n进度: {i+1}/{len(u_mats)} ",  'mean_infid:', mean_infid, 'max_infid', max_infid, end="")
    
print('\nmean_infid：', np.mean(mean_infid_list))
print('\nmax_infid：', np.mean(max_infid_list))


进度: 1/32  mean_infid: 0.3120028681405761 max_infid 0.43493070693417857
进度: 2/32  mean_infid: 0.33700488537752704 max_infid 0.5845074295204774

KeyboardInterrupt: 

# 用 abs 的自定义 MQLayer, 初始化方式为全 1 初始化。

In [2]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)

infid_list = [] # mean of infid
error_list = [] # max of infid
lr = 0.1
single_iter = 500 # 每个循环包含 100 次训练
max_iter = 40
for i in range(len(u_mats)):
    grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=0.05)  
    net = TrainOneStepCell(Quantum_net, opti)
    m = 0 # 用于计数 
    infid = 1
    while True:
        m += 1
        for j in range(single_iter):
            net(Tensor(train_x[j]), Tensor(train_y[i,j]))
            
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        infid_tem = 1-np.real(np.mean([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        infid = min(infid, infid_tem)
        if infid < 1e-5 or m >= max_iter:
            break
    infid_list.append(infid)
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    error_list.append(error)
    print(f"\n进度: {i+1}/{len(u_mats)} ",  'infid:', infid, 'error:', error, 'iter_num:', m*single_iter)
#     print(params)
print('\n mean of infid is：', np.mean(infid_list))
print('\n mean of error is：', np.mean(error_list))


进度: 1/32  infid: 1.8107737531636303e-13 error: 3.2218672174622043e-13 iter_num: 500

进度: 2/32  infid: 1.638911228951656e-12 error: 2.155164935402354e-12 iter_num: 1000

进度: 3/32  infid: 6.056693477862751e-07 error: 8.097765978876126e-07 iter_num: 1000

进度: 4/32  infid: 2.1347088050571728e-06 error: 3.922348002727283e-06 iter_num: 1000

进度: 5/32  infid: 9.747758156208874e-14 error: 1.3100631690576847e-13 iter_num: 500

进度: 6/32  infid: 3.0732875283545624e-06 error: 5.7128267495487606e-06 iter_num: 1500

进度: 7/32  infid: 5.942056846564014e-10 error: 7.782662292399323e-10 iter_num: 3000

进度: 8/32  infid: 2.05613304160579e-13 error: 3.4372504842394846e-13 iter_num: 500

进度: 9/32  infid: 1.525790693790441e-08 error: 2.224116224525119e-08 iter_num: 500

进度: 10/32  infid: 1.4189760477734126e-12 error: 1.9033663534173684e-12 iter_num: 1000

进度: 11/32  infid: 2.0250467969162855e-13 error: 2.8477220581635265e-13 iter_num: 3500

进度: 12/32  infid: 2.505910823291657e-08 error: 3.80648668141248e-08

In [4]:
print(infid_list)

[1.8107737531636303e-13, 1.638911228951656e-12, 6.056693477862751e-07, 2.1347088050571728e-06, 9.747758156208874e-14, 3.0732875283545624e-06, 5.942056846564014e-10, 2.05613304160579e-13, 1.525790693790441e-08, 1.4189760477734126e-12, 2.0250467969162855e-13, 2.505910823291657e-08, 1.6986412276764895e-13, 1.6753265441593612e-13, 1.5052958879380185e-11, 8.19366796633858e-12, 1.2323475573339238e-13, 2.786428195955537e-07, 3.3349264805249135e-06, 3.269758197532724e-08, 3.774820678259516e-09, 3.9190872769268026e-14, 1.83774966133754e-07, 1.520662510801074e-06, 3.531772652110021e-10, 1.5576429035490946e-13, 1.5156236327085182e-06, 1.162403506782539e-13, 3.667954828756592e-12, 1.2418954753457001e-12, 6.15423663141712e-08, 2.9363868633280887e-06]


In [5]:
print(error_list)

[3.2218672174622043e-13, 2.155164935402354e-12, 8.097765978876126e-07, 3.922348002727283e-06, 1.3100631690576847e-13, 5.7128267495487606e-06, 7.782662292399323e-10, 3.4372504842394846e-13, 2.224116224525119e-08, 1.9033663534173684e-12, 2.8477220581635265e-13, 3.80648668141248e-08, 2.297051437949449e-13, 2.311484337269576e-13, 2.121403053223503e-11, 1.1645906461410505e-11, 1.766364832178624e-13, 4.1575219345002523e-07, 4.669627582054581e-06, 6.487781767372525e-08, 5.339466979670249e-09, 6.716849298982197e-14, 2.5529041891569904e-07, 2.0835750229597494e-06, 5.186583384997334e-10, 2.0827783941967937e-13, 2.1022876081078223e-06, 1.8174350913113813e-13, 4.934497255248971e-12, 1.8276491431379327e-12, 8.736778889240071e-08, 5.331333469471566e-06]


In [11]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
# circ += gate_0('02').on(0)
# circ += gate_0('03').on(0)
# circ += gate_0('04').on(0)
# circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)

infid_list = [] # mean of infid
error_list = [] # max of infid
lr = 0.1
single_iter = 500 # 每个循环包含 100 次训练
max_iter = 7
for i in range(len(u_mats)):
    grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=0.05)  
    net = TrainOneStepCell(Quantum_net, opti)
    m = 0 # 用于计数 
    infid = 1
    while True:
        m += 1
        for j in range(single_iter):
            net(Tensor(train_x[j]), Tensor(train_y[i,j]))
            
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        infid_tem = 1-np.real(np.mean([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        infid = min(infid, infid_tem)
        if infid < 1e-5 or m >= max_iter:
            break
    infid_list.append(infid)
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    error_list.append(error)
    print(f"\n进度: {i+1}/{len(u_mats)} ",  'infid:', infid, 'error:', error, 'iter_num:', m*single_iter)
#     print(params)
print('\n mean of infid is：', np.mean(infid_list))
print('\n mean of error is：', np.mean(error_list))


进度: 1/32  infid: 0.4879651704707578 error: 0.69830996261144 iter_num: 3500

进度: 2/32  infid: 0.22674244437128 error: 0.30558891794989784 iter_num: 3500

进度: 3/32  infid: 0.4882782116240555 error: 0.8168808402585265 iter_num: 3500

进度: 4/32  infid: 0.23178333677566942 error: 0.3220099144380739 iter_num: 3500

进度: 5/32  infid: 0.025865331066970754 error: 0.03391961073694194 iter_num: 3500

进度: 6/32  infid: 0.3532569639554133 error: 0.4806290793444332 iter_num: 3500

进度: 7/32  infid: 0.2392877390582333 error: 0.38128277952624734 iter_num: 3500

进度: 8/32  infid: 0.002988577025838879 error: 0.005720781426915567 iter_num: 3500

进度: 9/32  infid: 0.06312278397257953 error: 0.0843506406550889 iter_num: 3500

进度: 10/32  infid: 0.26100334412228454 error: 0.3486808293748951 iter_num: 3500

进度: 11/32  infid: 0.5188022719320307 error: 0.8673122242111251 iter_num: 3500

进度: 12/32  infid: 0.5082539502635932 error: 0.7582591453772212 iter_num: 3500

进度: 13/32  infid: 0.276920876507846 error: 0.3672787

In [12]:
print(infid_list)

[0.4879651704707578, 0.22674244437128, 0.4882782116240555, 0.23178333677566942, 0.025865331066970754, 0.3532569639554133, 0.2392877390582333, 0.002988577025838879, 0.06312278397257953, 0.26100334412228454, 0.5188022719320307, 0.5082539502635932, 0.276920876507846, 0.36176306862191965, 0.009429280505722892, 0.006047970888024334, 0.13632510002122344, 0.07384480126362003, 0.2128052005467138, 0.0026864998634544834, 0.48615218581868413, 0.250425841183448, 0.0013502721246129168, 0.06009181010017073, 0.18641605389100147, 0.5550589880970995, 0.01832060521870771, 0.04226681827687462, 0.12563412915630456, 0.2213933191494063, 0.2994321984102001, 0.041598217991129616]


In [13]:
print(error_list)

[0.69830996261144, 0.30558891794989784, 0.8168808402585265, 0.3220099144380739, 0.03391961073694194, 0.4806290793444332, 0.38128277952624734, 0.005720781426915567, 0.0843506406550889, 0.3486808293748951, 0.8673122242111251, 0.7582591453772212, 0.36727878066656594, 0.5186300916504649, 0.012290204885242462, 0.007785680424798214, 0.1839327856931684, 0.11044570232581796, 0.3158282561101913, 0.01509111133455765, 0.7337429888526092, 0.35357158165095237, 0.0033517948254756513, 0.0835111846596086, 0.2587425995504664, 0.9514765985848216, 0.02499851377787221, 0.056989438727959585, 0.1822603166920922, 0.3007919230213256, 0.5436731937470266, 0.057901247554216106]


# 采用 MQLayer abs 方法

In [ ]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
# circ += gate_0('01').on(0)
# circ += gate_0('02').on(0)
# circ += gate_0('03').on(0)
# circ += gate_0('04').on(0)
# circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)

grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=0.05)  
net = TrainOneStepCell(Quantum_net, opti)

infid_list = [] # mean of infid
error_list = [] # max of infid
lr = 0.05
for i in range(len(u_mats)):
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))

    params = abs(Quantum_net.weight.asnumpy())
    final_state_0 = []
    for j in range(len(eval_x)):
        sim.reset()
        sim.set_qs(eval_x[j])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())

    infid_0 = 1-np.real(np.mean([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state_0), eval_y[i])]))
    if infid_0 < 1e-7:
        
        
        
    infid_list.append(infid)
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    error_list.append(error)
    print(f"\n进度: {i+1}/{len(u_mats)} ",  'infid:', infid, 'error:', error, end="")
#     print(params)

print('\n mean of infid is：', np.mean(infid_list))
print('\n mean of error is：', np.mean(error_list))

In [2]:
1-1e-7

0.9999999